In [ ]:
load_ext autoreload

In [ ]:
autoreload 2

In [ ]:
import numpy as np
import pyphi
import random
import makegridslibrary as me
import solarized
import matplotlib
import matplotlib.pyplot as plt
import itertools
from pypci import pci
import pickle


In [ ]:
bar_state = [0,0,0,0,0,0,0,0,1,1]
shu_state = [0,0,0,0,0,0,0,1,0,1]

N = 5

C_BGR_N5=me.make_cmgrid_withbackground(N,3)
# T_C_BGR_N5=me.make_tpm_withbackground(N,C_BGR_N5,3,.2)
T_C_BGR_N5=me.make_tpm_withbackground_gibs(N,C_BGR_N5,.25,[],2)


In [ ]:

results_filename = '/data/nsdm/pyphi/fivenodes_barVSshuffled_gridVSrandom_ENTROPY_WEDGE_DIFFSUM_T_0_3_D_2_MAIN_SAVED.pickle'
trange = [0, 2.75]

# results_filename = '/data/nsdm/pyphi/fivenodes_barVSshuffled_gridVSrandom_ENTROPY_WEDGE_DIFFSUM_T_0_1_D_2_MAIN_SAVED.pickle'
# trange = [0, 1.4]

with open(results_filename, 'rb') as f: 
    results = pickle.load(f)

(results_bar, results_shuffled) = (results[0], results[1])

ntemp = len(results_bar['Grid'])
# temperatures = [x/(ntemp-1) for x in range(ntemp)]
temperatures = np.linspace(trange[0], trange[1], ntemp)
    
do_plot = False
# do_plot = True

# ignore_homo = True
ignore_homo = False

weight_state = True
# weight_state = False

def operation(concepts, big_phi, operation_type = 'concepts * big_phi'):
    
#     operation_type = 'concepts'
#     operation_type = 'big_phi'
#     values = concepts
#     values = big_phi

    values = eval(operation_type)
#     print(values)
    
    return (np.sum(values), np.std(values))
#     return (np.sum(values), 0) 
#     return (np.mean(values), np.std(values))
#     return (np.mean(values), 0)
#     if isinstance(values, float):
#         return (1, 0) 
#     else:
#         return (len(values), 0) 

# (results_bar, results_shuffled) = pickle.load(, open(filename, 'wb'))


# network_labels = ['Grid']
network_labels = ['Grid', 'Random']
operation_types = ['concepts * big_phi', 'big_phi', 'concepts']

all_bar =  {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}
all_bar_std = {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}
all_shuffled = {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}
all_shuffled_std = {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}
all_matching = {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}
all_matching_std = {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}

all_matching_sum = {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}
all_matching_sum_std = {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}

for network_label in network_labels:
    
    if do_plot:
#         fig, axis = plt.subplots(int(len(temperatures)/2)+1, 2)
        fig, axis = plt.subplots(len(temperatures), 2)
        fig.set_size_inches(12,12)
        
    for operation_type in operation_types:
        
        for (itemp, temp) in enumerate(sorted(results_bar[network_label])):
            
            this_bar = results_bar[network_label][temp]
            number_bar = np.zeros(len(this_bar))
            number_bar_dict = {}
            matching_bar = np.zeros(len(this_bar))
            matching_bar_std = np.zeros(len(this_bar))
            for (ibar, bar_result) in enumerate(this_bar):

#                 print(bar_result)
                state_ok = len(this_bar[bar_result][1]) > 0

                if ignore_homo:
                    state_ok = state_ok & (sum(bar_result[:5]) > 0 and sum(bar_result[:5]) < len(bar_result[:5]))

                if state_ok:
                    number_bar[ibar] = len(this_bar[bar_result][1])
                    number_bar_dict[bar_result] = len(this_bar[bar_result][1])

                    (stat_1, stat_2) = operation(np.array(this_bar[bar_result][1]), this_bar[bar_result][0], operation_type)
                    (matching_bar[ibar], matching_bar_std[ibar]) = (stat_1, stat_2)

            if do_plot: # and itemp%2 == 0:
    #             idx = int(itemp/2)
                idx = itemp
                axis[idx, 0].hist(matching_bar)
                axis[idx, 0].axvline(x=np.mean(matching_bar), color='r')
                axis[idx, 0].set_title('mean: %2.5f' % np.mean(matching_bar))
                axis[idx, 0].set_ylabel('T: %2.2f' % temp)

            this_shuffled = results_shuffled[network_label][temp]
            number_shuffled = np.zeros(len(this_shuffled))
            number_shuffled_dict = {}
            matching_shuffled = np.zeros(len(this_bar))
            matching_shuffled_std = np.zeros(len(this_bar))

            for (ishuffled, shuffled_result) in enumerate(this_shuffled):

#                 print(shuffled_result)

                state_ok = len(this_shuffled[shuffled_result][1]) > 0

                if ignore_homo:
                    state_ok = state_ok & (sum(shuffled_result[:5]) > 0 and sum(shuffled_result[:5]) < len(shuffled_result[:5]))

                if state_ok:
                    number_shuffled[ishuffled] = len(this_shuffled[shuffled_result][1])
                    number_shuffled_dict[shuffled_result] = len(this_shuffled[shuffled_result][1])

                    (stat_1, stat_2) = operation(np.array(this_shuffled[shuffled_result][1]), this_shuffled[shuffled_result][0], operation_type)
                    (matching_shuffled[ishuffled], matching_shuffled_std[ishuffled]) = (stat_1, stat_2)

            if do_plot: # and itemp%2 == 0:
    #             idx = int(itemp/2)
                idx = itemp
                axis[idx, 1].hist(matching_shuffled)
                axis[idx, 1].axvline(x=np.mean(matching_shuffled), color='r')
                axis[idx, 1].set_title('mean: %2.5f' % np.mean(matching_shuffled))

#             print(itemp)
#             for (istate, state) in number_shuffled_dict:
#                 print('%d : ' % istate, end='')
#                 print (state)

            if weight_state:
                if network_label == 'Random':
                    matching_bar = matching_bar * bar_rand_prob_state[itemp,:]
                    matching_bar_std = matching_bar_std * bar_rand_prob_state[itemp,:]
                    matching_shuffled = matching_shuffled * shu_rand_prob_state[itemp,:]
                    matching_shuffled_std = matching_shuffled_std * shu_rand_prob_state[itemp,:]
                else:
                    matching_bar = matching_bar * bar_grid_prob_state[itemp,:]
                    matching_bar_std = matching_bar_std * bar_grid_prob_state[itemp,:]
                    matching_shuffled = matching_shuffled * shu_grid_prob_state[itemp,:]
                    matching_shuffled_std = matching_shuffled_std * shu_grid_prob_state[itemp,:]

            all_bar[network_label][operation_type][itemp] = np.mean(matching_bar)
            all_bar_std[network_label][operation_type][itemp] = np.sqrt(np.mean(matching_bar_std**2))
            all_shuffled[network_label][operation_type][itemp] = np.mean(matching_shuffled)
            all_shuffled_std[network_label][operation_type][itemp] = np.sqrt(np.mean(matching_shuffled_std**2))

            all_matching[network_label][operation_type][itemp] = np.mean(matching_bar) - np.mean(matching_shuffled)
            all_matching_std[network_label][operation_type][itemp] = np.sqrt(np.mean(matching_shuffled_std**2) + np.mean(matching_bar_std**2))

            all_matching_sum[network_label][operation_type][itemp] = np.sum(matching_bar) - np.sum(matching_shuffled)
            all_matching_sum_std[network_label][operation_type][itemp] = np.sqrt(np.sum(matching_shuffled_std**2) + np.sum(matching_bar_std**2))

    if do_plot:
        plt.tight_layout()
#         fig.suptitle(network_label + ' : ' + operation_type)
        fig.suptitle(network_label)
        plt.subplots_adjust(top=0.9)

if do_plot:
    plt.show()
    
# print(all_matching)